### 08 - Center bias predictions, 20231202

In this notebook we will generate validation metrics using the average bbox defined in the 07 notebook.

In [1]:
import torch
import pandas as pd
import numpy as np

from PIL import Image
from tqdm import tqdm
from torchvision.ops import box_iou

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

/home/nhormann/miniconda3/envs/rec-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
BASE_PATH = "../code/rec/"

In [41]:
def get_iou_column(df, bbox_pred_colname):
    df.loc[:, "bbox_raw"] = df.bbox_raw.apply(lambda v: np.resize(v, (1, 4)))
    df.loc[:, bbox_pred_colname] = df[bbox_pred_colname].apply(lambda v: np.resize(v, (1, 4)))
    df.loc[:, "iou"] = df.apply(lambda row: box_iou(torch.from_numpy(row[bbox_pred_colname]),
                                                    torch.from_numpy(row["bbox_raw"])), axis=1)
    df = df.astype({"iou": float})
    return df


def get_rec_counts(df_results, hits_col):
    counts = {
        hits_col: [],
        "counts": [],
        "rec_cls": []
    }
    for rec_cls in ["spatial", "ordinal", "relational", "intrinsic"]:
        counts["rec_cls"].append(rec_cls)
        mask_rec_cls = (df_results[rec_cls] == 1)
        counts[hits_col].append(df_results.loc[mask_rec_cls, hits_col].sum())
        counts["counts"].append(df_results.loc[mask_rec_cls, hits_col].shape[0])
    df_c = pd.DataFrame().from_dict(counts)
    df_c.loc[:, "exp_number"] = df_results.exp_number.unique()[0]
    df_c.loc[:, "exp_description"] = df_results.exp_description.unique()[0]
    df_c.loc[:, "hits_perc"] = round(df_c[hits_col] / df_c.counts * 100, 2)
    return df_c

### Read validation data from all available experiments results

Following dict's explanation:


exp_d:
```
key -> number of base experiment.

values -> list of lists[
    exp_number,
    validation_predictions_path(str),
    use_visual_pos_embeddings(boolean),
    use_visual_embeddings(boolean)
    ]
```

exp_descriptions:
```
key -> number of base experiment.
values -> description of the experiment.
```

In [45]:
exp_d = {
    "exp_003": [
        ["exp_003", "predictions_val.parquet", True, True],
    ],
}
exp_descriptions = {
    "exp_003": "Exp from the section 3.1 of the paper. But using an average bbox to compute the metrics.",
}

In [46]:
df_metrics_lst_25 = []
df_metrics_lst_50 = []
for exp_base, lst_exps in exp_d.items():
    exp_base_description = exp_descriptions[exp_base]
    for exp_number, exp_filename, use_visual_pos_emb, use_visual_emb in lst_exps:
        df_results = pd.read_parquet(f"../models/{exp_number}/{exp_filename}")
        df_results.loc[:, "exp_number"] = exp_number
        df_results.loc[:, "exp_description"] = exp_base_description
        df_results.loc[:, "avg_bbox"] = [[101.2846, 123.4821, 342.1320, 280.2839]] * df_results.shape[0] # coordinates defined in 07_NH
        df_results = get_iou_column(df_results, bbox_pred_colname="avg_bbox")
        df_results.loc[:, "hits_25"] = (df_results.iou > 0.25).astype(int)
        df_results.loc[:, "hits_50"] = (df_results.iou > 0.50).astype(int)
        df_m_25 = get_rec_counts(df_results, "hits_25")
        df_m_50 = get_rec_counts(df_results, "hits_50")
        df_metrics_lst_25.append(df_m_25)
        df_metrics_lst_50.append(df_m_50)
df_metrics_val_25 = pd.concat(df_metrics_lst_25, ignore_index=True)
df_metrics_val_50 = pd.concat(df_metrics_lst_50, ignore_index=True)

In [47]:
df_metrics_val_25

,hits_25,counts,rec_cls,exp_number,exp_description,hits_perc
0,656,4028,spatial,exp_003,Exp from the section 3.1 of the paper. But using an average bbox to compute the metrics.,16.29
1,10,90,ordinal,exp_003,Exp from the section 3.1 of the paper. But using an average bbox to compute the metrics.,11.11
2,194,1205,relational,exp_003,Exp from the section 3.1 of the paper. But using an average bbox to compute the metrics.,16.10
3,774,2290,intrinsic,exp_003,Exp from the section 3.1 of the paper. But using an average bbox to compute the metrics.,33.80


In [48]:
df_metrics_val_50

,hits_50,counts,rec_cls,exp_number,exp_description,hits_perc
0,38,4028,spatial,exp_003,Exp from the section 3.1 of the paper. But using an average bbox to compute the metrics.,0.94
1,0,90,ordinal,exp_003,Exp from the section 3.1 of the paper. But using an average bbox to compute the metrics.,0.00
2,13,1205,relational,exp_003,Exp from the section 3.1 of the paper. But using an average bbox to compute the metrics.,1.08
3,60,2290,intrinsic,exp_003,Exp from the section 3.1 of the paper. But using an average bbox to compute the metrics.,2.62
